# 무신사 크롤링

## 1. 링크 먼저 따오기

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time
import argparse
import random

In [4]:
def define_argparse():
    p = argparse.ArgumentParser()

    p.add_argument("--url", required=True)
    p.add_argument("--pages", type=int)
    p.add_argument("--type", required=True)

    config = p.parse_args()
    return config

상의 category URL
https://www.musinsa.com/categories/item/001

상의-셔츠/블라우스 category URL
https://www.musinsa.com/categories/item/001002

상의-맨투맨/스웨트셔츠 category URL
https://www.musinsa.com/categories/item/001005

바지 category URL
https://www.musinsa.com/categories/item/003

바지-데님 category URL
https://www.musinsa.com/categories/item/003002

바지-트레이닝/조거 팬츠 category URL
https://www.musinsa.com/categories/item/003004

**이처럼 각 url에는 "규칙"이 있습니다**

```python

```

In [6]:
# .py 버전에서 .sh와 연계하여 사용
# config = define_argparse()
# url = config.url

url = "https://www.musinsa.com/categories/item/001005"
options = webdriver.ChromeOptions()
# 탭 간 이동 활성화
options.add_argument("no-sandbox")

# 아무것도 안뜨게 조용하게 크롤링하기를 원할 때
#options.add_argument("headless")

In [8]:
links = []
goods_type = "top_sweatshirt"

# with as 구문
with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
    driver.get(url)
    driver.implicitly_wait(5)
    # 추후 누락된 친구들을 파악하기 위해 선언 후 초기화
    error_num = 0
    
    # for page_num in range(1, config.pages):
    # css selector는 보통 1부터 시작하기 때문에 0이 아닌 1로 시작
    for page_num in range(1, 3):
        
        # execute_script() : 인자로 받은 자바스크립트 코드를 웹브라우저에서 실행
        # 각 page_num의 페이지 번호로 이동
        driver.execute_script(f"switchPage(document.f1,{page_num});")
        
        # 웹페이지의 모든 요소가 로딩될때까지 기다림
        # 모든 요소가 로딩되면 넘어가고, 인자로 받은 숫자의 초 만큼 기다렸는데도 로딩이 안되면 그냥 go
        driver.implicitly_wait(5)
        
        # 한 페이지에 있는 90개의 옷 크롤링
        for item_num in range(1,91):
            # 크롤링하다보면 에러가 많이 발생함
            # 이에 대처하기 위해 try-except 구문을 애용하면 좋다.
            try:
                # find_element()
                # 인자로 받은 조건에 알맞은 html 요소 중 첫 번째 1개만 가져온다
                # return type : webelement (셀레니움 패키지 안에 정의되어있는 객체)
                
                # find_elements()
                # 인자로 받은 조건에 알맞은 html 요소 모든 것을 가져온다.
                # return type : list (list의 각 요소들은 webelement)
                
                link= driver.find_element(
                By.CSS_SELECTOR,
                f"#searchList > li:nth-child({item_num}) > div.li_inner > div.article_info > p.list_info > a")

                # get_attribute()
                # webelement 메소드로, 해당 html 요소의 href 요소를 긁어온다.
                # href : 하이퍼링크
                temp_link = link.get_attribute("href")
                links.append(temp_link)
                
            except:
                print(f"{page_num}page {item_num}th error!")
                error_num += 1
                continue
        
        # time.sleep()
        # 크롤링 프로그램을 잠깐 정지시킨다.
        # 이걸 안하면 IP 밴먹을 수 있음
        # 또한, 같은 주기로 계속 정지시켜도 IP 밴을 먹으니
        # random하게 정지시키는 주기를 변화해준다.
        time.sleep(random.randint(2,6))
            
        

print(f"{error_num} items omitted!")
link_df= pd.DataFrame(links)
link_df.to_csv(f"./data/{goods_type}.csv")




====== WebDriver manager ======
Current google-chrome version is 109.0.5414
Get LATEST chromedriver version for 109.0.5414 google-chrome
Driver [/Users/augustin/.wdm/drivers/chromedriver/mac64/109.0.5414.74/chromedriver] found in cache


0 items omitted!


## 2. 각 상품 정보 따오기

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time
import random
from urllib import request
import argparse
from tqdm import tqdm

In [10]:
def define_argparse():
    p = argparse.ArgumentParser()

    p.add_argument("--file_name", required=True)

    config = p.parse_args()
    return config

In [14]:
# .py 버전에서 .sh와 연계하여 사용
# config = define_argparse()
# url = config.url
# df = pd.read_csv(f"./data/{config.file_name}.csv")

df = pd.read_csv(f"./data/top_sweatshirt.csv")
url_list = df.iloc[:,1]
options = webdriver.ChromeOptions()
# 탭 간 이동 활성화
options.add_argument("no-sandbox")
# options.add_argument("headless")



product_num_list = []
product_name_list = []
brand_list = []
year_sold_list = []
like_list = []
rate_list = []
price_list = []
tag_list = []


with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
    omitted_num = 0
    # 테스트를 위해 3개만 뽑아봅시다
    for url in tqdm(url_list[3:6]):
        
        # 크롤링을 진행하다보면 어떤 상품에는 해당 요소가 존재하지 않을 수 있음
        # 이런 경우에는 존재하지 않는 요소를 None값 취급하고 뽑을지,
        # 아예 해당 상품을 뽑지 않을지를 선택해야함
        # 본 예제에서는 뽑지 않고 지나가는것을 선택함
        # 이를 위해 try-except 구문 활용
        try:
                driver.get(url)
                driver.implicitly_wait(5)
                
                # https://www.musinsa.com/app/goods/2096933
                # url의 규칙에 따라 맨 마지막에는 해당 상품의 고유 번호가 담겨있음 
                product_num = url.split("/")[-1]
                
                # 상품 이름
                product_name =  driver.find_element(
                        By.CSS_SELECTOR,
                        "#page_product_detail > div.right_area.page_detail_product > div.right_contents.section_product_summary > span > em")
                
                # 상품 브랜드
                brand = driver.find_element(
                        By.CSS_SELECTOR,
                        "#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(1) > p.product_article_contents > strong > a")
                
                # 1년 판매량
                year_sold = driver.find_element(
                        By.CSS_SELECTOR,
                        "#sales_1y_qty")

                # 좋아요 개수
                like = driver.find_element(
                        By.CSS_SELECTOR,
                        f"#product-top-like > p.product_article_contents.goods_like_{product_num} > span")

                # 평점
                rate = driver.find_element(
                        By.CSS_SELECTOR,
                        "#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(6) > p.product_article_contents > a > span.prd-score__rating")
                
                # 가격
                price = driver.find_element(
                        By.CSS_SELECTOR,
                        "#goods_price")
                
                # 이미지를 뽑기 위해 이미지 링크 추출
                image_url = driver.find_element(
                        By.CSS_SELECTOR,
                        "#detail_bigimg > div > img").get_attribute("src")
                
                # 해당 상품의 태그 추출
                # 여기서는 태그가 여러개이기 때문에 find elements
                # 즉, return은 list의 형태(혹은 iterable)
                tags = driver.find_elements(
                    By.CSS_SELECTOR,
                    "#product_order_info > div.explan_product.product_info_section > ul > li.article-tag-list.list a"
                )
                
                temp_tag_list = []
                # tags에 list의 형태(혹은 iterable)가 담겨있으므로, 반복문을 돌려서 하나씩 추출
                for tag in tags:
                # tag 맨 앞에 있는 # 제거
                    temp_tag_list.append(tag.text[1:])

                # temp_tag_list에 들어있는 해체된 태그들을
                # 추후 다루기 쉽게 문자열로 변환
                tag_str = ",".join(temp_tag_list)
                

        # 위 과정에서 문제가 발생했을 경우 (특정 요소가 해당 상품에 존재하지 않을 경우)
        # 예외 처리 한 다음 누락된 개수 count 추가
        # 이후 continue로 다음 상품
        except:
            omitted_num +=1
            continue
        
        # 상품에서 추출한 요소들을 리스트에 append
        # (단, 이 과정은 with as 구문으로 파일 입출력으로 대체해도 무방함)
        # 저는 이게 편해서...ㅎㅎ
        
        product_num_list.append(product_num)
        product_name_list.append(product_name.text)
        brand_list.append(brand.text)
        year_sold_list.append(year_sold.text)
        like_list.append(like.text)
        rate_list.append(rate.text)
        price_list.append(price.text)
        tag_list.append(tag_str)
        
        # 아까 추출해놓은 이미지 url을 이용하여
        # request.urlretrieve() 메소드를 이용하여 이미지 저장
        # 이때 이미지 파일 이름은 상품에 부여된 고유 번호로 설정
        request.urlretrieve(image_url, f"./data/image/{product_num}.jpg")
        
        # ip 밴 방지
        time.sleep(random.uniform(0,1))

        # 매 반복마다 csv를 새로 만드는 이유
        # 다 끝나고 csv를 만들면, 예상치 못한 오류 (보통은 각 리스트마다 길이가 다른 경우 -> 이 경우 추출이 잘못된거에요)로 인해
        # 뽑긴 뽑았는데 그걸 저장을 못하는 매우 슬픈 일이 발생할 수 있음
        # 따라서, 매 반복마다 csv를 저장하도록 하면 중간에 끊기더라도 이전까지 뽑은 정보들을 살릴 수 있다는 이점이 존재함
        # 다만, 이 방법은 시스템에 매우 큰 부하를 불러일으킨다는 단점이 있음
        
        product_df = pd.DataFrame({"product_num" :product_num_list,
                            #main_category_list,
                            #sub_category_list,
                            "product_name" : product_name_list,
                            "brand" : brand_list,
                            "year_sold" : year_sold_list,
                            "like" : like_list,
                            "rate" : rate_list,
                            "price" : price_list,
                            "tag" : tag_list
                            })
    
        product_df.to_csv(f"./data/top_sweatshirt_info.csv")
        

    print()
    print(f"top_sweatshirt : {omitted_num} omitted!")




====== WebDriver manager ======
Current google-chrome version is 109.0.5414
Get LATEST chromedriver version for 109.0.5414 google-chrome
Driver [/Users/augustin/.wdm/drivers/chromedriver/mac64/109.0.5414.74/chromedriver] found in cache
100%|██████████| 3/3 [00:36<00:00, 12.09s/it]


top_sweatshirt : 0 omitted!


## 3. 각 상품의 리뷰 따오기

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import pandas as pd
import time
import random
from urllib import request
import argparse
from tqdm import tqdm
import re

In [22]:
def define_argparse():
    p = argparse.ArgumentParser()

    p.add_argument("--file_name", required=True)

    config = p.parse_args()
    return config

In [25]:
# .py 버전에서 .sh와 연계하여 사용
# config = define_argparse()
# url = config.url
# df = pd.read_csv(f"./data/{config.file_name}.csv")


df = pd.read_csv(f"./data/top_sweatshirt_info.csv")
url_list = df["product_num"]

options = webdriver.ChromeOptions()
# 탭 간 이동 활성화
options.add_argument("no-sandbox")
# options.add_argument("headless")


review_list = []
star_list = []
product_num_list = []


with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
    omitted_num = 0
    for num in tqdm(url_list):
        try:
            # 상품 고유번호 이용하여 url 생성
            url = f"https://www.musinsa.com/app/goods/{num}"
            driver.get(url)
            driver.implicitly_wait(5)
            
            # ()안에 존재하는 총 리뷰 개수 추출하기 위해 정규표현식 사용
            inside_bracket = re.compile('\((.*?)\)')
            print("loading complete")
            # 스타일 후기 개수
            style = driver.find_element(
                By.CSS_SELECTOR,
                "#estimate_style"
            )
            style_temp = inside_bracket.findall(style.text)[0]
            print(style_temp)
            style_temp = style_temp.replace(",", "")
            # 문자열이기 때문에 정수형으로 변환
            style_num = int(style_temp)

            # 상품 후기 개수
            photo = driver.find_element(
                By.CSS_SELECTOR,
                "#estimate_photo"
            )
            
            photo_temp = inside_bracket.findall(photo.text)[0]
            print(photo_temp)
            photo_temp = photo_temp.replace(",", "")
            # 문자열이기 때문에 정수형으로 변환
            photo_num = int(photo_temp)

            # 일반 후기 개수
            goods = driver.find_element(
                By.CSS_SELECTOR,
                "#estimate_goods"
            )
            goods_temp = inside_bracket.findall(goods.text)[0]
            print(goods_temp)
            goods_temp = goods_temp.replace(",", "")
            # 문자열이기 때문에 정수형으로 변환
            goods_num = int(goods_temp)
            print(goods_num)

            # 각 리뷰 타입
            review_type_list = [style, photo, goods]
            time.sleep(random.uniform(0,2))
            
            # 각 리뷰 20개씩 없으면 패스
            if style_num < 3 or photo_num < 3 or goods_num < 3:
                omitted_num += 1
                print("리뷰 부족")

                continue
            
            # 리뷰 정렬 조작하기 위해 정렬 요소 추출
            sort = driver.find_element(
                By.CSS_SELECTOR,
                "#reviewSelectSort"
            )
            
            # 토글, 드롭다운 등 다양한 요소를 조작하기 위헤 
            #selenium.webdriver.support.ui.Select 사용
            sort_select = Select(sort)

            # 각 리뷰 종류 클릭으로 이동 (각 리뷰 당 40개 추출)
            # 리뷰 버튼에는 따로 자바스크립트 요소가 없기 때문에 click() 메소드로 조작
            for review_type in review_type_list:
                if review_type != style:
                    review_type.click()
                # IP 밴 방지
                time.sleep(random.uniform(0,1))
                
                # 높은 평점 순으로 정렬
                # 조작하고 싶은 값의 value를 찾은 뒤,
                # select_by_value() 메소드로 이를 조작
                sort_select.select_by_value("goods_est_desc")
                
                # 각 리뷰 별 높은 평점 순 20개 추출
                for page in range(1, 3):
                    if page != 1:
                        driver.execute_script(f"ReviewPage.goPage({page});")
                    time.sleep(random.uniform(0,1))
                    for i in range(1,11):
                        
                        # 리뷰 텍스트 추출
                        review = driver.find_element(
                            By.CSS_SELECTOR,
                            f"#reviewListFragment > div:nth-child({i}) > div.review-contents > div.review-contents__text"
                        )
                        review_txt = review.text
                        review_txt = review_txt.replace("\n", " ")

                        # 별점 추출
                        # 이때, 별점이 그림으로만 있고 텍스트로 따로 보이지 않는다
                        # 이런 경우에는 텍스트로 전환할 수 있는 요소를 찾아서 해결해보자
                        star_raw = driver.find_element(
                            By.CSS_SELECTOR,
                            f"#reviewListFragment > div:nth-child({i}) > div.review-list__rating-wrap > span > span > span"
                        )
                        
                        # 해당 요소의 style 항목을 보면,
                        # 별 1개는 20, 2개는 40의 규칙이 보인다. (style="width: 100%")
                        # 이를 이용해보자
                        
                        # get_attribute()로 해당 요소의 style의 값 불러오기
                        star = star_raw.get_attribute("style")
                        
                        # width: 제거 위해 split
                        star_temp = star.split(": ")[-1]
                        
                        # %과 ; 제거
                        star_temp = star_temp.replace("%", "")
                        star_temp = star_temp.replace(";", "")
                        
                        # 남겨진 텍스트를 정수형으로 바꾼 뒤, 20으로 나눠준다
                        star_result = int(star_temp) / int(20)
                        product_num_list.append(num)
                        review_list.append(review_txt)
                        star_list.append(star_result)


                    time.sleep(random.uniform(0,1))
        


                # 낮은 평점 순으로 정렬
                # 조작하고 싶은 값의 value를 찾은 뒤,
                # select_by_value() 메소드로 이를 조작
                sort_select.select_by_value("goods_est_asc")
                
                # 각 리뷰 별 낮은 평점 순 20개 추출
                for page in range(1, 3):
                    if page != 1:
                        driver.execute_script(f"ReviewPage.goPage({page});")
                    time.sleep(random.uniform(0,1))
                    for i in range(1,11):
                        
                        # 리뷰 텍스트 추출
                        review = driver.find_element(
                            By.CSS_SELECTOR,
                            f"#reviewListFragment > div:nth-child({i}) > div.review-contents > div.review-contents__text"
                        )
                        review_txt = review.text
                        review_txt = review_txt.replace("\n", " ")

                        # 별점 추출
                        # 이때, 별점이 그림으로만 있고 텍스트로 따로 보이지 않는다
                        # 이런 경우에는 텍스트로 전환할 수 있는 요소를 찾아서 해결해보자
                        star_raw = driver.find_element(
                            By.CSS_SELECTOR,
                            f"#reviewListFragment > div:nth-child({i}) > div.review-list__rating-wrap > span > span > span"
                        )
                        
                        # 해당 요소의 style 항목을 보면,
                        # 별 1개는 20, 2개는 40의 규칙이 보인다. (style="width: 100%")
                        # 이를 이용해보자
                        
                        # get_attribute()로 해당 요소의 style의 값 불러오기
                        star = star_raw.get_attribute("style")
                        
                        # width: 제거 위해 split
                        star_temp = star.split(": ")[-1]
                        
                        # %과 ; 제거
                        star_temp = star_temp.replace("%", "")
                        star_temp = star_temp.replace(";", "")
                        
                        # 남겨진 텍스트를 정수형으로 바꾼 뒤, 20으로 나눠준다
                        star_result = int(star_temp) / int(20)
                        product_num_list.append(num)
                        review_list.append(review_txt)
                        star_list.append(star_result)


                    time.sleep(random.uniform(0,1))
                    

                product_df = pd.DataFrame({"product_num" :product_num_list,
                                            "review" : review_list,
                                            "star" : star_list
                                            })
                
                product_df.to_csv(f"./data/top_sweatshirt_review.csv")
        except:
            omitted_num += 1
            continue
        
    print(f"{omitted_num} omitted!")




====== WebDriver manager ======
Current google-chrome version is 109.0.5414
Get LATEST chromedriver version for 109.0.5414 google-chrome
Driver [/Users/augustin/.wdm/drivers/chromedriver/mac64/109.0.5414.74/chromedriver] found in cache
  0%|          | 0/3 [00:00<?, ?it/s]

loading complete
2,582
3,832
13,248
13248


 33%|███▎      | 1/3 [00:47<01:35, 47.61s/it]

loading complete
857
1,040
4,180
4180


 67%|██████▋   | 2/3 [01:13<00:34, 34.85s/it]

loading complete
555
633
2,064
2064


100%|██████████| 3/3 [01:41<00:00, 33.90s/it]


0 omitted!
